In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/PostDocProject/keras-gcn/'
!python setup.py install
from __future__ import print_function
import tensorflow
import keras
from keras.layers import Input, Dropout, Embedding, Dense, Conv1D, GlobalMaxPooling1D, Reshape, Lambda, Concatenate, LayerNormalization, Add
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.regularizers import l2
import scipy
from scipy import io
import kegra
from kegra.layers.graph import GraphConvolution
from kegra.utils import *
import time
import tensorflow as tf
from config import *
support=1
print(keras.__version__)
print(tf.__version__)
!python --version

Mounted at /content/drive
/content/drive/My Drive/PostDocProject/keras-gcn
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, us

In [2]:
# @title Default title text
!pip install rdkit
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
import os
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle
import collections
from collections import OrderedDict
from matplotlib.pyplot import cm
import numpy as np
from scipy.io import savemat, loadmat
import math

CHARPROTSET = { "A": 1, "C": 2, "B": 3, "E": 4, "D": 5, "G": 6,
                "F": 7, "I": 8, "H": 9, "K": 10, "M": 11, "L": 12,
                "O": 13, "N": 14, "Q": 15, "P": 16, "S": 17, "R": 18,
                "U": 19, "T": 20, "W": 21,
                "V": 22, "Y": 23, "X": 24,
                "Z": 25 }

CHARPROTLEN = 25

CHARCANSMISET = { "#": 1, "%": 2, ")": 3, "(": 4, "+": 5, "-": 6,
             ".": 7, "1": 8, "0": 9, "3": 10, "2": 11, "5": 12,
             "4": 13, "7": 14, "6": 15, "9": 16, "8": 17, "=": 18,
             "A": 19, "C": 20, "B": 21, "E": 22, "D": 23, "G": 24,
             "F": 25, "I": 26, "H": 27, "K": 28, "M": 29, "L": 30,
             "O": 31, "N": 32, "P": 33, "S": 34, "R": 35, "U": 36,
             "T": 37, "W": 38, "V": 39, "Y": 40, "[": 41, "Z": 42,
             "]": 43, "_": 44, "a": 45, "c": 46, "b": 47, "e": 48,
             "d": 49, "g": 50, "f": 51, "i": 52, "h": 53, "m": 54,
             "l": 55, "o": 56, "n": 57, "s": 58, "r": 59, "u": 60,
             "t": 61, "y": 62}

CHARCANSMILEN = 62

CHARISOSMISET = {"#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, ".": 2,
                "1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, "7": 38, "6": 6,
                "9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43,
                "D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13,
                "O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50, "T": 17, "W": 51,
                "V": 18, "Y": 52, "[": 53, "Z": 19, "]": 54, "\\": 20, "a": 55, "c": 56,
                "b": 21, "e": 57, "d": 22, "g": 58, "f": 23, "i": 59, "h": 24, "m": 60,
                "l": 25, "o": 61, "n": 26, "s": 62, "r": 27, "u": 63, "t": 28, "y": 64}

CHARISOSMILEN = 64


## ######################## ##
#
#  Encoding Helpers
#
## ######################## ##

#  Y = -(np.log10(Y/(math.pow(math.e,9))))

def one_hot_smiles(line, MAX_SMI_LEN, smi_ch_ind):
    X = np.zeros((MAX_SMI_LEN, len(smi_ch_ind))) #+1

    for i, ch in enumerate(line[:MAX_SMI_LEN]):
        X[i, (smi_ch_ind[ch]-1)] = 1

    return X #.tolist()

def one_hot_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
    X = np.zeros((MAX_SEQ_LEN, len(smi_ch_ind)))
    for i, ch in enumerate(line[:MAX_SEQ_LEN]):
        X[i, (smi_ch_ind[ch])-1] = 1

    return X #.tolist()


def label_smiles(line, MAX_SMI_LEN, smi_ch_ind):
    X = np.zeros(MAX_SMI_LEN)
    for i, ch in enumerate(line[:MAX_SMI_LEN]): #    x, smi_ch_ind, y
        X[i] = smi_ch_ind[ch]

    return X #.tolist()

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
    X = np.zeros(MAX_SEQ_LEN)

    for i, ch in enumerate(line[:MAX_SEQ_LEN]):
        X[i] = smi_ch_ind[ch]

    return X #.tolist()




# Set the following ones properly
f = open('/content/drive/My Drive/PDB/Y_PDB.txt', 'r')    # file which contain label of PDB data  (uploaded in git repository)
f1 = open('/content/drive/My Drive/PDB/name_PDB.txt', 'r')  # file which contain the name of samples in pdb dataset  (uploaded in git repository)
path_pdb = '/content/drive/My Drive/PDB/v2016'  # the path of dataset


XD = []  # matrix which stores drugs
XT = []  # matrix which stores proteins

y_value = []
y_unit = []
y_real = []
flag = []

!pip install Bio

for i, l in enumerate(f):
    print(i)
    try:
        tmp = f.readline()
        value = tmp.split('>=')
        if len(value)<2:
            value = tmp.split('>')
            if len(value)<2:
               value = tmp.split('<=')
               if len(value)<2:
                  value = tmp.split('<')
                  if len(value)<2:
                     value = tmp.split('=')
                     if len(value)<2:
                        value = tmp.split('~')
                        if len(value)<2:
                           print(value)

        y_value.append(value[1][:-3])
        y_unit.append(value[0])
        if value[0] == 'IC50':
            #   flag.append(0)
            #if value[0] == 'Kd':
            #   flag.append(1)
            #if value[0] == 'Ki':
            #   flag.append(2)
            print(i)


            folder = f1.readline()
            folder = folder.split('.')[0]
            print(path_pdb+'/'+folder+'/'+folder+'_ligand.sdf')
            m = Chem.MolFromMolFile(path_pdb+'/'+folder+'/'+folder+'_ligand.sdf')
            print(path_pdb+'/'+folder+'/'+folder+'_ligand.sdf')
            #print(Chem.MolToSmiles(m))
            #print('here1')

            #print('here2')

            from Bio.PDB import PDBParser
              # this is the module described below
            d3to1 = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
            'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N',
            'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W',
            'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M', 'HOH': 'B'}
            # read
            sloppyparser = PDBParser(
                PERMISSIVE=True#, structure_builder=xpdb.SloppyStructureBuilder()
            )
            structure = sloppyparser.get_structure("MD_system", path_pdb + '/'+ folder+'/'+folder+'_protein.pdb')
            #print(structure.get_chains(structure.get_models()))
            struct = PDBParser(QUIET=True).get_structure('tmp', path_pdb + '/' + folder+'/'+folder+'_protein.pdb')
            #print(struct)
            count=0
            for model in structure:
                for chain in model:
                    if count==0:
                        seq = []
                        for residue in chain:
                            seq.append(d3to1[residue.resname])
                        #print(seq)
                        count=count+1

            XD.append(label_smiles(Chem.MolToSmiles(m), 100, CHARISOSMISET))

            XT.append(label_sequence(seq, 1000, CHARPROTSET))
            if value[1][-3:-1] == 'pM':
               y_real.append(float(value[1][:-3])* (0.001))
            if value[1][-3:-1] == 'uM':
               y_real.append(float(value[1][:-3])* (10^(3)))
            if value[1][-3:-1] == 'mM':
               y_real.append(float(value[1][:-3])* (10^6))
            if value[1][-3:-1] == 'fM':
               y_real.append(float(value[1][:-3])* (0.000001))
            if value[1][-3:-1] == 'nM':
               y_real.append(float(value[1][:-3]))

            if len(y_real) > 500:
              break;
    except:
        continue

print(type(y_value))
print(type(y_unit))
print(set(y_unit))
print(type(y_real))
print(np.sum(np.array(flag)==0))
print(np.sum(np.array(flag)==1))
print(np.sum(np.array(flag)==2))
print(len(y_real))
print(len(XD))
print(len(XT))

y_real = -(np.log10(np.array(y_real)/(math.pow(10,9)))) #-np.log(y_real/(10^9))

pdb={'XT': XT, 'XD': XD, 'Y': y_real}
savemat('pdb_xt.mat',pdb)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 42.1 MB/s eta 0:00:00


In [ ]:
from scipy.io import savemat, loadmat
tmp = loadmat('pdb_xt.mat')
XD = tmp['XD']
XT = tmp['XT']
y_real = tmp['Y'].T

########################################
#  prepare  data                       #
########################################

# choose the appropriate flag
train_drugs = np.array(XD[0:350])
train_prots = np.array(XT[0:350])
train_Y = np.array(y_real[0:350])

threshold = np.median(train_Y)

val_drugs = np.array(XD[350:])
val_prots = np.array(XT[350:])
val_Y = np.array(y_real[350:])

#train_Y = np.reshape(train_Y,(train_Y.shape[1],))
#val_Y = np.reshape(val_Y,(val_Y.shape[1],))


num_train_samples = train_Y.shape[0]
num_val_samples = val_Y.shape[0]

###################################################


# Define parameters
DATASET = 'cora'
FILTER = 'localpool'  # 'chebyshev'
MAX_DEGREE = 2  # maximum polynomial degree
SYM_NORM = True  # symmetric (True) vs. left-only (False) normalization
NB_EPOCH = 0
PATIENCE = 10  # early stopping patience


def cindex_score(y_true, y_pred):

    g = tf.subtract(tf.expand_dims(y_pred, -1), y_pred)
    g = tf.cast(g == 0.0, tf.float32) * 0.5 + tf.cast(g > 0.0, tf.float32)

    f = tf.subtract(tf.expand_dims(y_true, -1), y_true) > 0.0
    f = tf.compat.v1.matrix_band_part(tf.cast(f, tf.float32), -1, 0)

    g = tf.reduce_sum(tf.multiply(g, f))
    f = tf.reduce_sum(f)

    return tf.where(tf.equal(g, 0), 0.0, g/f) #select

def dot_batch(inp):
    import tensorflow as tf
    return tf.math.sigmoid(tf.tensordot(inp[0][0:100],inp[1],axes=(1, 1)))

def dot_batch2(inp):
    import tensorflow as tf
    a=inp[0]
    b=inp[1]
    return tf.tensordot(a,b,axes=(0, 0))

# Compound Feature Extractor


Drug_input = Input(shape=(smiles_max_len,), dtype='int32',name='drug_input')
Protein_input = Input(shape=(protein_max_len,), dtype='int32',name='protein_input')
G = [Input(batch_shape=(None, None))]#, batch_shape=(None, None))]

encode_smiles = Embedding(input_dim=smiles_dict_len+1, output_dim = embedding_size, input_length=smiles_max_len,name='smiles_embedding')(Drug_input)
encode_smiles = Conv1D(filters=num_filters, kernel_size=smiles_filter_lengths,  activation='relu', padding='valid',  strides=1, name='conv1_smiles')(encode_smiles)
encode_smiles = Conv1D(filters=num_filters*2, kernel_size=smiles_filter_lengths,  activation='relu', padding='valid',  strides=1, name='conv2_smiles')(encode_smiles)
encode_smiles = Conv1D(filters=num_filters*3, kernel_size=smiles_filter_lengths,  activation='relu', padding='valid',  strides=1, name='conv3_smiles')(encode_smiles)



encode_protein = Embedding(input_dim=protein_dict_len+1, output_dim = embedding_size, input_length=protein_max_len, name='protein_embedding')(Protein_input)
encode_protein = Conv1D(filters=num_filters, kernel_size=protein_filter_lengths,  activation='relu', padding='valid',  strides=1, name='conv1_prot')(encode_protein)
encode_protein = Conv1D(filters=num_filters*2, kernel_size=protein_filter_lengths,  activation='relu', padding='valid',  strides=1, name='conv2_prot')(encode_protein)
encode_protein = Conv1D(filters=num_filters*3, kernel_size=protein_filter_lengths,  activation='relu', padding='valid',  strides=1, name='conv3_prot')(encode_protein)

encode_protein = GlobalMaxPooling1D()(encode_protein)
encode_smiles = GlobalMaxPooling1D()(encode_smiles)



protein_desc_len=96


X_in =Concatenate()([encode_protein,encode_smiles])  # this correspond with node descriptor
X_in= LayerNormalization()(X_in)

model_Feat = Model(inputs=[Drug_input, Protein_input], outputs=[X_in])

input_adj = Input(shape=(num_filters*6,))
FC1 = Dense(1024, activation='relu', name='dense1')(input_adj)
FC2 = Dropout(0.1)(FC1)
FC2 = Dense(1024, activation='relu', name='dense2')(FC2)
FC2 = Dropout(0.1)(FC2)
FC2 = Dense(512, activation='relu', name='dense3')(FC2)
# And add a logistic regression on top
G1 = Dense(1, kernel_initializer='normal',  name='dense4')(FC2) # if you want train model for active/inactive set activation='sigmoid'

model_Adj = Model(inputs=input_adj, outputs=[G1])


H = Dropout(0.5)(X_in)
H = GraphConvolution(512, support, activation='relu', kernel_regularizer=l2(5e-4))([X_in]+G) #, kernel_regularizer=l2(5e-4)
#H = Dropout(0.5)(H)
#H = GraphConvolution(192, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = GraphConvolution(256, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)

Y = GraphConvolution(192, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G) #activation='sigmoid',, name='main_out'

Y = Add()([Y,X_in])
Y = Dense(1024, activation='relu')(Y)
Y = Dropout(0.1)(Y)
Y = Dense(1024, activation='relu')(Y)
Y = Dropout(0.1)(Y)
Y = Dense(512, activation='relu')(Y)
# And add a logistic regression on top
Y = Dense(1)(Y)

model_Semi = Model(inputs=[Drug_input, Protein_input,G], outputs=Y)

# Compile model

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
]

def weighted_bce(y_true, y_pred):
  y_true = keras.backend.cast(y_true,"float32")
  weights = (y_true * 18.) + 1.
  bce = keras.backend.binary_crossentropy(y_true, y_pred)
  weighted_bce = keras.backend.mean(bce * weights)
  return weighted_bce
def bce(y_true, y_pred):
  y_true = keras.backend.cast(y_true,"float32")

  bce = keras.backend.binary_crossentropy(y_true[0,:], y_pred[0,:])
  weighted_bce = keras.backend.mean(bce)
  return weighted_bce

out_ = model_Adj(model_Feat([Drug_input,Protein_input]))
model2 = Model(inputs=[Drug_input,Protein_input], outputs=out_)
model2.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate_feat_model),metrics=['mse',cindex_score]) #weighted_bce


import keras.backend as K
def mean_squared_error_w(y_true, y_pred):
    mses = K.mean(K.square(y_pred - y_true), axis = -1)
    std_of_pred = K.std(y_pred)
    std_of_true = K.std(y_true)
    mses_std =K.square(std_of_pred - std_of_true)
    #const = K.mean(mses, axis = -1) + (std_of_mses * 0.5)
    #mask = K.cast(K.less(K.mean(K.square(y_pred - y_true), axis=-1), const), dtype = "float32")
    return mses_std+mses
def mean_squared_error_w1(y_true, y_pred):
    mses = K.mean(K.square(y_pred - y_true), axis = -1)
    std_of_mses = K.std(mses)
    const = K.mean(mses, axis = -1) + (std_of_mses * 0.5)
    mask = K.cast(K.less(K.mean(K.square(y_pred - y_true), axis=-1), const), dtype = "float32")
    return mask * K.mean(K.square(y_pred - y_true), axis=-1)

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR') # precision-recall curve
]
model_Semi.compile(loss=['mse'], optimizer=Adam(learning_rate=learning_rate_GCN),metrics=['mse',cindex_score])
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999



###########################################
#  Design Generator                       #
###########################################
from scipy.sparse import csr_matrix
def generate_data(batch_size,flag):
  i_c=0
  while True:
        input1 = []
        input2 = []

        output1 = []
        batch_counter=0

        if (i_c+batch_size)>num_train_samples:
          i_c= i_c - ((i_c+batch_size) - num_train_samples )
        elif (i_c+batch_size)==num_train_samples:
          i_c = 0

        G_all = model_Feat.predict([np.array(train_drugs),np.array(train_prots)],batch_size=batch_size)

        in_drug=train_drugs[i_c:i_c+batch_size]
        in_prots=train_prots[i_c:i_c+batch_size]
        #G = model2.predict([np.array(input1[0:batch_size]),np.array(input2[0:batch_size])],batch_size=batch_size)


        idx = np.concatenate((range(0,i_c),range(i_c+1+batch_size,train_drugs.shape[0])))
        idx=idx.astype(int)
        #################################################################
        ## Retrieval step
        #################################################################
        G_all = np.array(G_all)
        paired_dist = scipy.spatial.distance.cdist(G_all[i_c:(i_c+batch_size), :], G_all[idx, :])
        idx_out = np.argsort(paired_dist, axis = 1)
        Y_sorted  = np.reshape([train_Y[idx[index2]]>threshold  for index in idx_out for index2 in index], (batch_size, len(idx)))
        local_pattern = Y_sorted
        out = scipy.signal.convolve2d(local_pattern, [[1, -1]], mode='same')
        idx_ = []
        for ii in range(out.shape[0]):
          for jj in range(out.shape[1]):
            if out[ii, jj] != 0:
              idx_.append(idx_out[ii, jj])

        idx_ = np.unique(np.array(idx_))
        if len(idx_) >= 10*batch_size:
           pindex = np.random.choice(idx_.shape[0], 10*batch_size, replace=False)
           pindex = idx_[pindex]
        else:
           pindex = np.random.choice(idx_.shape[0], (10*batch_size), replace=True)
           pindex = idx_[pindex]
        ###############################################################################
        #pindex = np.random.choice(idx.shape[0], 10*batch_size, replace=False)
        input1 = np.concatenate((in_drug,train_drugs[idx[pindex]]), axis=0)
        input2 = np.concatenate((in_prots,train_prots[idx[pindex]]), axis=0)

        output1 = np.concatenate((train_Y[i_c:i_c+batch_size],train_Y[idx[pindex]]), axis=0)
        output1 = np.reshape(output1,(10*batch_size+batch_size,1))
        print(np.sum(train_Y[idx[pindex]]>threshold))
        G = model2.predict([np.array(input1[0:batch_size]),np.array(input2[0:batch_size])],batch_size=batch_size)

        output1_ = (output1>threshold)
        output1_ = output1_.astype(int)
        tmpp=np.matmul(output1_,np.transpose(output1_))+np.matmul(1-output1_,np.transpose(1-output1_))


        G1=np.array(G)
        G1 = (G1>threshold)
        G1 = G1.astype(int)

        G1 = np.matmul(G1,np.transpose(output1_))+np.matmul(1-G1,np.transpose(1-output1_))

        Adj = np.vstack((np.reshape(G1[0:batch_size,batch_size:],(batch_size,10*batch_size)),tmpp[batch_size:,batch_size:]))
        Adj = np.hstack((np.reshape(G1,(batch_size+10*batch_size,batch_size)),Adj)) # [0:100,:]

        Adj = Adj+np.eye(Adj.shape[0])
        d = np.diag(np.power(np.array(Adj.sum(1)), -0.5).flatten(), 0)
        Adj = Adj.dot(d).transpose().dot(d)


        """print(np.array(input1).shape)
        print(np.array(input2).shape)
        print(Adj.shape)
        print(np.array(output1).shape)
        print(batch_size)"""

        i_c=i_c+batch_size
        model_Semi.fit([np.array(input1),np.array(input2), Adj],np.array(output1),sample_weight=np.concatenate((np.zeros((batch_size,)),np.ones((10*batch_size,)))),
                 batch_size=10*batch_size+batch_size, epochs=10, shuffle=False, )

        if flag==1:
           yield [np.array(input1),np.array(input2), Adj], np.array(output1), np.concatenate((np.zeros((batch_size,)),np.ones((10*batch_size,))))
        else:
           yield [np.array(input1),np.array(input2), Adj], output1_[0:batch_size]


def generate_data_val(batch_size_test, batch_size,flag):
  i_c=0
  while True:
        input1 = []
        input2 = []

        output1 = []
        batch_counter=0

        if i_c>=num_val_samples:
          i_c=0


        in_drug=val_drugs[i_c:i_c+batch_size_test]
        in_prots=val_prots[i_c:i_c+batch_size_test]

        G_all = model_Feat.predict([np.array(train_drugs),np.array(train_prots)],batch_size=batch_size)

        idx = np.concatenate((range(0,i_c),range(i_c+1+batch_size_test,train_drugs.shape[0])))
        idx=idx.astype(int)
        #################################################################
        ## Retrieval step
        #################################################################
        G_all = np.array(G_all)
        paired_dist = scipy.spatial.distance.cdist(G_all[i_c:(i_c+batch_size_test), :], G_all[idx, :])
        idx_out = np.argsort(paired_dist, axis =1)
        Y_sorted  = np.reshape([train_Y[idx[index2]]>threshold  for index in idx_out for index2 in index], (batch_size, len(idx)))
        local_pattern = Y_sorted
        out = scipy.signal.convolve2d(local_pattern, [[1, -1]], mode='same')
        idx_ = []
        for ii in range(out.shape[0]):
          for jj in range(out.shape[1]):
            if out[ii, jj] != 0:
              idx_.append(idx_out[ii, jj])

        idx_ = np.unique(np.array(idx_))
        if len(idx_) >= 10*batch_size:
           pindex = np.random.choice(idx_.shape[0], 10*batch_size, replace=False)
           pindex = idx_[pindex]
        else:
           pindex = np.random.choice(idx_.shape[0], (10*batch_size), replace=True)
           pindex = idx_[pindex]
        #########################################################################
        #pindex = np.random.choice(idx.shape[0], 10*batch_size, replace=False)
        #print(pindex)

        input1 = np.concatenate((in_drug,train_drugs[idx[pindex]]), axis=0)
        input2 = np.concatenate((in_prots,train_prots[idx[pindex]]), axis=0)


        output1 = np.concatenate((val_Y[i_c:i_c+batch_size_test],train_Y[idx[pindex]]), axis=0)
        output1 = np.reshape(output1,(batch_size+10*batch_size,1))

        G = model2.predict([np.array(input1[0:batch_size]),np.array(input2[0:batch_size])],batch_size=batch_size)

        output1_ = (output1>threshold)
        output1_ = output1_.astype(int)
        tmpp=np.matmul(output1_,np.transpose(output1_))+np.matmul(1-output1_,np.transpose(1-output1_))


        G1=np.array(G)
        G1 = (G1>threshold)
        G1 = G1.astype(int)

        G1 = np.matmul(G1,np.transpose(output1_))+np.matmul(1-G1,np.transpose(1-output1_))

        Adj = np.vstack((np.reshape(G1[0:batch_size,batch_size:],(batch_size,10*batch_size)),tmpp[batch_size:,batch_size:]))
        Adj = np.hstack((np.reshape(G1,(10*batch_size+batch_size,batch_size)),Adj)) # [0:100,:]

        Adj = Adj+np.eye(Adj.shape[0])
        d = np.diag(np.power(np.array(Adj.sum(1)), -0.5).flatten(), 0)
        Adj = Adj.dot(d).transpose().dot(d)

        print(np.array(input1).shape)
        print(np.array(input2).shape)
        print(Adj.shape)
        print(np.array(output1).shape)
        print(batch_size)

        i_c=i_c+batch_size_test
        """for kk in range(0,10):
            model_Semi.fit([np.array(input1),np.array(input2), Adj],np.array(output1),sample_weight=np.concatenate((np.zeros((batch_size1,)),np.ones((10*batch_size1,)))),
                 batch_size=1100, epochs=1, shuffle=False)
        """
        if flag==1:
           yield [np.array(input1),np.array(input2), Adj], np.array(output1), np.concatenate((np.zeros((batch_size,)),np.ones((10*batch_size,))))
        else:
           yield [np.array(input1),np.array(input2), Adj], output1_[0:batch_size]

num_train_samples = len(train_drugs)

for epoch in range(0, NB_EPOCH+1):
    #batch_size1=100
    model_Adj.trainable=True
    model_Feat.trainable=True

    #model2.fit(([np.array(train_drugs),np.array(train_prots) ]), np.array(train_Y), batch_size=512, epochs=10,
    #                         validation_data=( ([np.array(val_drugs), np.array(val_prots) ]), np.array(val_Y)),  shuffle=False )
    import scipy
    #model_Adj.trainable=False
    model_Feat.trainable=False
    batch_size = 256
    history = model_Semi.fit_generator(generator=generate_data(batch_size,1),
              steps_per_epoch = num_train_samples/batch_size, epochs=20, shuffle=False, verbose=1)#,validation_data=generate_data_val(batch_size1), validation_steps=50)


2/2 [==============================] - 1s 27ms/step
1521
1/1 [==============================] - 1s 1s/step
Epoch 1/10
1/1 [==============================] - 21s 21s/step - loss: 58.3597 - mse: 62.5030 - cindex_score: 0.5184
Epoch 2/10
1/1 [==============================] - 1s 795ms/step - loss: 57.8988 - mse: 61.9985 - cindex_score: 0.5282
Epoch 3/10
1/1 [==============================] - 1s 822ms/step - loss: 57.3718 - mse: 61.4338 - cindex_score: 0.5225
Epoch 4/10
1/1 [==============================] - 1s 814ms/step - loss: 56.8979 - mse: 60.9071 - cindex_score: 0.5102
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 56.4353 - mse: 60.4001 - cindex_score: 0.5091
Epoch 6/10
1/1 [==============================] - 1s 822ms/step - loss: 55.9363 - mse: 59.8660 - cindex_score: 0.5097
Epoch 7/10
1/1 [==============================] - 1s 1s/step - loss: 55.4482 - mse: 59.3426 - cindex_score: 0.5030
Epoch 8/10
1/1 [==============================] - 1s 922ms/step - loss: 54

KeyboardInterrupt: ignored

In [ ]:

# Test Phase
i_c=0
batch_size_test = 1
batch_size = 1
preds= []
num_neighbours_test = 100
while True:
    input1 = []
    input2 = []
    output1 = []
    batch_counter=0

    if i_c>=num_val_samples:
        break

    in_drug=val_drugs[i_c:i_c+batch_size_test]
    in_prots=val_prots[i_c:i_c+batch_size_test]

    G_all = model_Feat.predict([np.array(train_drugs), np.array(train_prots)], batch_size=batch_size)

    idx = np.concatenate((range(0, i_c), range(i_c + 1 + batch_size_test, train_drugs.shape[0])))
    idx = idx.astype(int)


    #################################################################
    ## Retrieval step
    #################################################################
    G_all = np.array(G_all)
    paired_dist = scipy.spatial.distance.cdist(G_all[i_c:(i_c + batch_size_test), :], G_all[idx, :])
    idx_out = np.argsort(paired_dist, axis=1)
    Y_sorted = np.reshape([train_Y[idx[index2]] > threshold for index in idx_out for index2 in index],
                          (batch_size, len(idx)))
    local_pattern = Y_sorted
    out = scipy.signal.convolve2d(local_pattern, [[1, -1]], mode='same')
    idx_ = []
    for ii in range(out.shape[0]):
        for jj in range(out.shape[1]):
            if out[ii, jj] != 0:
                idx_.append(idx_out[ii, jj])

    idx_ = np.unique(np.array(idx_))
    if len(idx_) >= 10 * batch_size:
        pindex = np.random.choice(idx_.shape[0], 10 * batch_size, replace=False)
        pindex = idx_[pindex]
    else:
        pindex = np.random.choice(idx_.shape[0], (10 * batch_size), replace=True)
        pindex = idx_[pindex]
    #########################################################################

    pindex = np.random.choice(idx.shape[0], num_neighbours_test, replace=False)
    #print(pindex)

    input1 = np.concatenate((in_drug,train_drugs[idx[pindex]]), axis=0)
    input2 = np.concatenate((in_prots,train_prots[idx[pindex]]), axis=0)


    output1 = np.concatenate((val_Y[i_c:i_c+batch_size_test],train_Y[idx[pindex]]), axis=0)
    output1 = np.reshape(output1,(num_neighbours_test+batch_size_test,1))

    G = model2.predict([np.array(input1[0:batch_size_test]),np.array(input2[0:batch_size_test])],batch_size=batch_size_test)

    output1_ = (output1>threshold)
    output1_ = output1_.astype(int)
    tmpp=np.matmul(output1_,np.transpose(output1_))+np.matmul(1-output1_,np.transpose(1-output1_))


    G1=np.array(G)
    G1 = (G1>threshold)
    G1 = G1.astype(int)

    G1 = np.matmul(G1,np.transpose(output1_))+np.matmul(1-G1,np.transpose(1-output1_))

    Adj = np.vstack((np.reshape(G1[0:batch_size_test,batch_size_test:],(batch_size_test,num_neighbours_test)),tmpp[batch_size_test:,batch_size_test:]))
    Adj = np.hstack((np.reshape(G1,(num_neighbours_test+batch_size_test,batch_size_test)),Adj)) # [0:100,:]

    Adj = Adj+np.eye(Adj.shape[0])
    d = np.diag(np.power(np.array(Adj.sum(1)), -0.5).flatten(), 0)
    Adj = Adj.dot(d).transpose().dot(d)



    i_c=i_c+batch_size_test
    model_Semi.fit([np.array(input1),np.array(input2), Adj],np.array(output1),sample_weight=np.concatenate((np.zeros((batch_size_test,)),np.ones((num_neighbours_test,)))),
            batch_size=num_neighbours_test+batch_size_test, epochs=num_epoch_GCN_inference, shuffle=False)

    preds.append(model_Semi.predict([np.array(input1),np.array(input2), Adj], batch_size=num_neighbours_test+batch_size_test)[0])
print(preds)

preds_m=preds
m=keras.metrics.MeanSquaredError()
m.update_state(preds_m,val_Y)
print("MSE= {:.4f}".format(m.result().numpy()))
print(cindex_score(val_Y,preds_m))


1/1 [==============================] - 0s 491ms/step
Epoch 1/20
1/1 [==============================] - 0s 308ms/step - loss: 0.4639 - mse: 0.2590 - cindex_score: 0.8622
Epoch 2/20
1/1 [==============================] - 0s 58ms/step - loss: 0.4309 - mse: 0.2329 - cindex_score: 0.8797
Epoch 3/20
1/1 [==============================] - 0s 59ms/step - loss: 0.4486 - mse: 0.2462 - cindex_score: 0.8714
Epoch 4/20
1/1 [==============================] - 0s 44ms/step - loss: 0.4335 - mse: 0.2388 - cindex_score: 0.8775
Epoch 5/20
1/1 [==============================] - 0s 44ms/step - loss: 0.4118 - mse: 0.2144 - cindex_score: 0.8729
Epoch 6/20
1/1 [==============================] - 0s 43ms/step - loss: 0.4101 - mse: 0.2094 - cindex_score: 0.8888
Epoch 7/20
1/1 [==============================] - 0s 42ms/step - loss: 0.4387 - mse: 0.2474 - cindex_score: 0.8656
Epoch 8/20
1/1 [==============================] - 0s 38ms/step - loss: 0.4295 - mse: 0.2331 - cindex_score: 0.8765
Epoch 9/20
1/1 [==========